In [ ]:
import pandas as pd 
import numpy as np
import glob
import os
import re
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive

Informal ReadMe: 
- My top priority is not scalabilty (yet) 
- 

In [ ]:
# Michael's Advice: 
    # One df per patient
    # Pivot? Get dummies?
    # Rolling Window   !!! Ben said 3 second window is best
    
# JM's Advice: 
    # Write a function to bring in all the data
    # Have it make a patient column and doc column
    # But getting an MVP could be super useful too. 
        # So if you want to try the experiment on pt 1, that's good too. 

# Experimentation with patient #1

CPG #1

In [ ]:
CPG1 = pd.read_csv('Barretts_Analysis/CPG/001 CPG.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = CPG1.iloc[0]
header = header.str.replace('\n', ' ')
CPG1 = CPG1.rename(columns = header)
CPG1 = CPG1[1:]

CPG1 = CPG1.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

CPG1 = CPG1[CPG1["Meas Type"].str.contains("MII")]
CPG1 = CPG1.rename(columns={'Meas Start Time': 'CPG Start Time'})

In [ ]:
Auto1 = pd.read_csv('Barretts_Analysis/Autoscan/001 Autoscan.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = Auto1.iloc[0]
header = header.str.replace('\n', ' ')
Auto1 = Auto1.rename(columns = header)
Auto1 = Auto1[1:]

Auto1 = Auto1.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

Auto1 = Auto1[Auto1["Meas Type"].str.contains("MII")]
Auto1 = Auto1.rename(columns={'Meas Start Time': 'Auto Start Time'})

In [ ]:
Auto1.shape

# Test 1: merge

# Test 2: Concatenating CPG + Auto - Patient 1

In [ ]:
TestCPGAuto2 = pd.concat([CPG1, Auto1], axis = 1).fillna('').drop(columns = {'Meas Type', 'Body Position', 'Contents'})

In [ ]:
TestCPGAuto2.shape

In [ ]:
# If you have a Doc column again and stack vertically, you could sort by start time. 
# Probably need Start Day and Start Time re.split

In [ ]:
TestCPGAuto2.info()

# Test 3: Concatenate vertically - CPG, Auto - Patient 1

In [ ]:
CPG1b = pd.read_csv('Barretts_Analysis/CPG/001 CPG.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = CPG1b.iloc[0]
header = header.str.replace('\n', ' ')
CPG1b = CPG1b.rename(columns = header)
CPG1b = CPG1b[1:]

CPG1b = CPG1b.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

CPG1b = CPG1b[CPG1b["Meas Type"].str.contains("MII")]
CPG1b['Doc'] = 'CPG'

Auto1b = pd.read_csv('Barretts_Analysis/Autoscan/001 Autoscan.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = Auto1b.iloc[0]
header = header.str.replace('\n', ' ')
Auto1b = Auto1b.rename(columns = header)
Auto1b = Auto1b[1:]

Auto1b = Auto1b.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

Auto1b = Auto1b[Auto1b["Meas Type"].str.contains("MII")]
Auto1b['Doc'] = 'Auto'

In [ ]:
TestCPGAuto3 = pd.concat([CPG1b, Auto1b]).fillna('').drop(columns = {'Meas Type', 'Body Position', 'Contents'})

In [ ]:
TestCPGAuto3

In [ ]:
TestCPGAuto3.Doc.value_counts()

Remember: 
    # If you have a Doc column again and stack vertically, you could sort by start time. 
    # Probably need Start Day and Start Time re.split

* Could I write a function to read in the files and label them with the patient number? And Doc name? 

* Do I still need to split the start time? Should it be a datetime object? 

* Could you do ML to find out who is predicting better? Reflux = 1, No Reflux = 0

# Try to bring in all data & label accordingly 

In [ ]:
import pandas as pd
import glob

path = 'Barretts_Analysis/'
next_path = glob.glob(path + "*")
#all_files = glob.glob(next_path + "/*.csv")



# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# command + ? = BIG comment

li = []

for some_path in next_path: 
    #print(some_path) 
    if some_path == 'Barretts_Analysis/AI': 
        continue
    else: 
        for file in glob.glob(some_path + "/*.csv"):
            #print(file)
            df = pd.read_csv(file, index_col=None, header=0, skiprows = 7).fillna('-')
            header = df.iloc[0]
            header = header.str.replace('\n', ' ')
            df = df.rename(columns = header)
            df = df[1:]
            df = df.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])
            df = df[df["Meas Type"].str.contains("MII")]
            #df['patient_number'] = file.replace('/', ' ')
            df['z_filename'] = file.replace(' ', '-')
            #df['z_filename'] = df['z_filename'].dropna()
            #print(df.head())
            li.append(df)
        
frame = pd.concat(li, axis=0, ignore_index=True)
# command + ? = BIG comment

But I don't have my doc name or patient number....!

In [ ]:
frame.z_filename.value_counts()

In [ ]:
frame.z_filename[0]

In [ ]:
frame[['patient_num','doc', 'None', 'None']] = frame.z_filename.str.split("-",expand=True) 

In [ ]:
frame[['folder1', 'doctor', 'patient']] = frame.patient_num.str.split('/', expand = True).replace('')

In [ ]:
frame['patient2'] = frame.patient_num.str.extract('(\d+)').astype(int)

In [ ]:
frame = frame.drop(columns = {'Meas Type', 'Body Position', 'Contents', 'patient_num', 'doc', 'None', 'folder1', 'z_filename', 'patient', '-'})

In [ ]:
frame

In [ ]:
# Split meas start time and convert to datetime. Convert to 24 hour? 

In [ ]:
frame.patient2.value_counts()

In [ ]:
frame.doctor.value_counts()

In [ ]:
frame[['Meas Start Day', 'Meas Start Time']] = frame['Meas Start Time'].str.split("/",expand=True)

In [ ]:
frame.info()

In [ ]:
# maybe 

# frame['Meas Start Time 2'] = pd.to_datetime(frame['Meas Start Time'], format = '%H:%M:%S.%f')
#frame['Meas Start Time 2'] = pd.to_datetime(frame['Meas Start Time'])

In [ ]:
frame

In [ ]:
frame.info()

In [ ]:
#frame['Meas Start Time 2'].str.strip('T00:00:00')

In [ ]:
frame

# Dibran's code for shifting

In [ ]:
frame['Meas Start TimeStamp'] = pd.to_datetime(frame['Meas Start Time'])
frame['3_second_shift'] = frame['Meas Start TimeStamp'] + timedelta(seconds = -3)
frame['3_second_shift_forward'] = frame['Meas Start TimeStamp'] + timedelta(seconds = 3)
frame['3_second_target'] = 0
new_frame = frame.groupby('doctor')

In [ ]:
%%time
filtered_rows = pd.DataFrame([])
for k in new_frame.groups.keys():
    data = new_frame.get_group(k)
    for x, y in data.iterrows():
        if y['MII Reflux'] =='Reflux':
            the_only_hope = data[(data['Meas Start TimeStamp'] > y['3_second_shift'])&
                              (data['Meas Start TimeStamp'] < y['Meas Start TimeStamp'])]
        else: 
            continue
    filtered_rows = filtered_rows.append(the_only_hope)

In [ ]:
frame.head()

In [ ]:
frame['Meas Start Day'] = frame['Meas Start Day'].astype('int64')

In [ ]:
frame = frame.sort_values(["patient2", "Meas Start Day", "Meas Start Time"], ascending = (True, True, True))
frame.head()
# consider using 4 seconds instead of 3

In [ ]:
frame = frame.reset_index(drop=True)

## sort by (patient?) then by meas time , write a func starting on first row and iterate through rows, for each row, see if next row occurs within three sec. If yes, label meas. if not, increase by one

## dont forget same day, same patient 

if current_item_time is +- 3 sec of previous_item_time: 
    and Meas Start Day == previous Meas Start Day:
        label Measurement (n)
    
    

In [ ]:
for item, frame in df['Meas Start TimeStamp'].iteritems():
    if next <= '00:00:03.000':
        print frame
        2020-05-14 16:12:40.400

In [ ]:
[(df[1].ge(df[1].shift())) & df[1].le(df[1].shift(-1))]

In [ ]:
def read_row(df):

    if df['Meas Start Time'].current == df['Meas Start Time'].next:
        df['Meas No.']' = 1
    elif df['Meas Start Time'].next <= 3s of df['Meas Start Time'].current:
        df['Meas No.'] = '1'
    else df['Meas No.'].value = df['Meas No.'].next



In [ ]:
def read_row_2(df):
    for row in df.iterrows:
        if (row['Meas Start TimeStamp'].next - row['Meas Start TimeStamp'].current).dt.total_seconds() <= 3.00:
            row['Meas No.'] = 1
        else: 
            row['Meas No.'] = row['Meas No.'] + 1          

Nested loop, use groupbys instead of for loops (grupby patient and day)
Pandas.cut to do a groupby arange on time
for each group, iterate over the groups and add labels 

In [ ]:
by_pt_dy = frame.groupby(['patient2', 'Meas Start Day'])

for patient, startday in by_pt_dy: 
    print(f"First 2 entries for {patient!r}")
    print("------------------------")
    print(startday.head(2), end="\n\n")

In [ ]:
frame[frame['Meas Start Day'] == 2]